<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">pyGROWAT2D</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);"><b style=color:red;>GRO</b>und<b style=color:red;>WAT</b>er</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
<td><img style="height: 150px;" src="images/pyGROWAT2D.png"></td>
</tr>
</table>

----
# `pyGROWAT2D`

GROundWATer2D, a program package for flow in porous rocks.

----
# Groundwater flow

This notebook is a summary of all relevant equations we will use in this project,
plus additional derivation for those more interested in the theory.

## Groundwater-flow equation

The program `GROWAT1D` (GROundWATer2D) solves an **advection-diffusion-reaction** equation
without **diffusion** ($\vec{u} \cdot \nabla c=0$) and without **reaction** ($M=R=0$) in a two-dimensional
modeling domain for a porous aquifer under steady-state or transient flow conditions:
$$
\begin{array}{rcl}
 S \frac{\partial h}{\partial t} & = & - \nabla \cdot \vec{F} \\
 \vec{F} &=& - K \nabla h
\end{array}
$$
- The first equation is a conservation equation for flow, with 
$h$ [m] the hydraulic head,
$S$ [1/m] the specific storage,
$\vec{F}$ the Dary-flow velocity, 
$\nabla$ [1/m] the nabla operator, and 
$t$ [s] time.
- The second equation is the **Darcy law** for laminar flow, 
with $K$ [m/s] the hydraulic conductivity.

The **Darcy law** has been derived from laboratory experiments on water flow through a vertical column,
which was filled with different porous materials (e.g. [wikipedia](https://en.wikipedia.org/wiki/Darcy%27s_law))

**Notes:**

- From here on we can directly jump to the next notebook about the numerical implementation
of the groundwater-flow equation ...
- ... or we derive the Dary law from fundamental computational fluid dynamics:

----
----
## Computational fluid dynamics
### Continuity

The **continuity equation** describes the flux of a quantity through a medium, e.g. for **concentration** $c$:
$$\fbox{$
\frac{\partial c}{\partial t}
= - \vec{u} \cdot \nabla c
- \nabla \cdot \vec{F}
+ M +R
$}$$
with:
- $c$ [mol/m$^3$] a quantity (e.g. temperature, concentration, pressure, ...)
- $t$ [s] time
- $\vec{u}$ [m/s] velocity
- $\vec{F}$ [mol/m$^2$/s] flux
- $M$ [mol/m$^3$/s] source term
- $R$ [mol/m$^3$/s] reaction term
- $\nabla$ [1/m] nabla operator

From this general **advection-diffusion-reaction** equation we can define equations for
- water pressure (head)
- water velocities
- temperature
- concentration
- dissolution

### Equation of motion
The **equation of motion** describes the velocity induced by surface and/or body forces:
$$\fbox{$
\rho \frac{\partial \vec{u}}{\partial t}
+ \rho \left( \vec{u} \cdot \nabla \right) \vec{u}
= \nabla \cdot \bar{\sigma}
- \rho \vec{g}
$}$$
with:
- $\rho$ [kg/m$^3$] density,
- $\vec{u}$ [m/s] velocity,
- $t$ [s] time,
- $\nabla$ [1/m] Nabla operator,
- $\mathbb{\sigma}$ [Pa] Cauchy stress tensor,
- $\vec{g}$ [m/s] gravitational acceleration.

### Mass-conservation equation
The **mass-conservation equation** describes the continuity of mass:
$$\fbox{$
\frac{\partial \rho}{\partial t}
+ \nabla \cdot \left( \rho \vec{u} \right) 
= 0
$}$$
with:
- $\rho$ [kg/m$^3$] density,
- $\vec{u}$ [m/s] velocity,
- $t$ [s] time,
- $\nabla$ [1/m] Nabla operator.

### Stress tensor and strain-rate tensor
The **Cauchy stress tensor** is defined as:
$$\fbox{$
\mathbb{\sigma} = -p \mathbb{1} + 2 \eta \dot{\mathbb{\epsilon}} 
$}$$
with:
- $p$ [Pa] pressure,
- $\eta$ [Pas] dynamic viscosity,
- $\dot{\mathbb{\epsilon}}$ [1/s] strain-rate tensor,
- $\mathbb{1}$ [-] the unity tensor.

The **strain-rate tensor** is defined as:
$$\fbox{$
\dot{\mathbb{\epsilon}} = \frac{1}{2} \left[ \nabla \vec{u} + (\nabla \vec{u})^T \right]
$}$$
with:
- $\vec{u}$ [m/s] velocity.

----
## Navier-Stokes equation

Combining the equation of motion, Cauchy stress and strain-rate, we arrive at the 
**Navier-Stokes equation** for a viscous fluid:
$$\fbox{$
\rho \frac{\partial \vec{u}}{\partial t}
+ \rho \left( \vec{u} \cdot \nabla \right) \vec{u}
= - \nabla p
+ \eta \Delta \vec{u}
- \rho \vec{g}
$}$$

----
## Pipe-flow model
We use the **Navier-Stokes equation** to derive the Darcy law by defining flow through pore spaces or fissures with a 
simple **circular pipe-flow model**:

<img src="images/poiseuille_flow.jpg" style="width:12cm;">

The circular pipe of defined length $l$ [m] has a given cross-sectional area $A$ [m$^2$] and radius $R$ [m], and flow is driven
by a pressure difference, with high pressure, $p+dp$ [Pa] on the left, and low pressure $p$ on the right side.
We define a cylindrical coordinate system, with the $z$-axis in flow direction and the $r$-axis perpendicular to
flow, and employ the Navier-Stokes equation  in a simplified form: 
- We assume stationary flow (${{\partial \vec{u}} \over {\partial t}}=0$), 
- and we neglect both the advective and the gravitional terms, $\rho (\vec{u} \cdot \nabla) \vec{u}=0$ and $\rho g=0$:
$$
\nabla p =
\eta \Delta \vec{u}
$$

From the three equation stated above, we use the derivative in $z$-direction, as the pressure gradient is only in the $z$-direction,
and we rewrite the Laplacian operator of the vectorial velocity $u_i$ with the corresponding term:
$$
   \frac{\partial^2 u_z}{\partial r^2}
 + \frac{1}{r} \frac{\partial^2 u_z}{\partial \Phi^2}
 + \frac{\partial^2 u_z}{\partial z^2}
 + \frac{1}{r} \frac{\partial u_z}{\partial r}
 = \frac{1}{\eta} \frac{\partial p}{\partial z}
$$
with $(r,z,\Phi)^T$ the coordinates given in the cylindrical coordinate system.
As the fluid velocity component $u_z$ only depends on $r$, two of the four derivatives on the right-hand side
vanish, and we can reformulate the equation to:
$$
 \frac{1}{r} \frac{\partial}{\partial r} \left( r \frac{\partial u_z}{\partial r} \right)
 = \frac{1}{\eta} \frac{\partial p}{\partial z}
$$
Integrating this equation twice with respect to $r$ and applying the boundary conditions
$v_z(r=\pm R)=0$ then results in the fluid velocity component:
$$
 u_z(r) = \frac{1}{4\eta} \frac{\partial p}{\partial z} \left( r^2 - R^2 \right)
$$
which is a **parabolic** flow profile.

----
## Hagen-Poiseuille equation
We proceed integrating the velocity profile $u_z$ over the cross-sectional area $A$ to obtain the **flowrate** $Q$ [m$^3$/s]:
$$
 Q = \int\limits_{0}^{2\pi} \int\limits_{0}^{R} v_z(r) r dr d\Phi 
   = \frac{2\pi}{4\eta} \frac{\partial p}{\partial z} \int\limits_{0}^{R} \left( r^3 - r R^2 \right) dr
   = \frac{2\pi}{4\eta} \frac{\partial p}{\partial z} \big[ \frac{r^4}{4} - \frac{r^2}{2} R^2 \big]_0^R
   = \frac{2\pi}{4\eta} \frac{\partial p}{\partial z} \big[ \frac{R^4}{4} - \frac{R^2}{2} R^2 \big]
   = -\pi R^2 \frac{R^2}{8\eta} \frac{\partial p}{\partial z} 
$$
which results in the Hagen-Poiseuille flow for a circular pipe:
$$\fbox{$
 Q = - A \frac{\kappa}{\eta} \frac{\partial p}{\partial z}
$}$$
with 
- $A=\pi R^2$ [m$^2$] the cross-sectional area, and
- $\kappa=\frac{R^2}{8}$ [m$^2$] the permeability.

Finally, we divide the flowrate $Q$ through the corss-ectional area $A$, $F=\frac{Q}{A}$, and replace the pressure $p=\rho g h$, 
with $\rho$ [kg/m$^3$] the density of the fluid, $g$ [m/s$^2$] the gravitational acceleration,
and $h$ [m] the **hydraulic head**:
$$
F = - \underbrace{\kappa \frac{\rho g}{\eta}}_{K} \frac{\partial h}{\partial z}
$$
with $K$ [m/s] the hydraulic conductivity.

This is actually the one-dimensional form of the **Darcy law** we introduced above as laboratory result!

<src img="images/poiseuille_flow.jpg" style="width:12cm;">

----